In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
from functions import complex_correlation, colorize

# for Jupyter lab
%matplotlib widget
# for Jupyter notebook
# %matplotlib notebook

data_folder = 'data'

SMALL_SIZE = 16
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [2]:
from matplotlib import colors

def get_color_map(n):
    cdict = {'red':   ((0.0,  0.3, 0.3),
                       (1.0,  1.0, 1.0)),

             'green': ((0.0,  0.1, 0.1),
                       (1.,  0., 0.)),

             'blue':  ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0))}
    cmap = colors.LinearSegmentedColormap('custom', cdict, N=n)
    return cmap

# 1. First load the params and data

## Load parameters

In [4]:
save_path = '../Data/'
with open(save_path + 'param.json','r') as f:
    param = json.load(f)
    print('Succesfully loaded params of no pressure exp from json file')

Succesfully loaded params of no pressure exp from json file


## Load theoretical modes

In [6]:
b = np.load(save_path+ 'conversion_matrices.npz')
modes_in = b.f.modes_in
modes_in_full = np.kron(np.array([[1,0],[0,1]]),modes_in)#np.concatenate((modes_in,modes_in),axis = 0)
n = np.int(np.sqrt(modes_in.shape[1]))
modes_out = b.f.modes_out
modes_out_full = np.kron(np.array([[1,0],[0,1]]),modes_out)#np.concatenate((modes_out,modes_out),axis = 0)
m = np.int(np.sqrt(modes_out.shape[1]))

## Load the TMs

In [9]:
TM_modes = []
for i, pos in enumerate(param['Translation']['steps_pressure']):
    filename = save_path + 'TM_modes_'+ str(pos) + '.npz'
    #np.savez_compressed(filename, TM_corr_modes=TM_corr_modes)
    b = np.load(filename)
    TM_modes.append(b.f.TM_corr_modes)
TM_modes = np.array(TM_modes)

# 4. Channels robust to deformation

In [10]:
def pca_inv(H,n1,n0=0):
    u,s,vh = np.linalg.svd(H)
    print(s[:10])
    s2 = np.zeros_like(s)
    s2[n0:n1] = 1./s[n0:n1]
    return vh.transpose().conjugate() @ np.diag(s2) @ u.transpose().conjugate() 

def get_WS(H0,H1,H2):
#     H1 = remove_global_phase(H1)
#     H2 = remove_global_phase(H2)
#     H0 = remove_global_phase(H0)
    dH = (H2 - H0)*0.5
    WS = -1j*pca_inv(H1,110)@dH
#     WS = -1j*dH
#     WS = -1j * np.linalg.pinv(H1,rcond = 0.5) @ dH
#     WS = -1j * H1.transpose().conjugate() @ dH
#     u,s,v = np.linalg.svd(np.linalg.pinv(TM1,rcond = 0.5))
#     plt.figure(),plt.plot(s,'+')
    return WS

def get_WS_symm(H0,H1,H2):
    WS = get_WS(H0,H1,H2)
    WS_symm = WS + WS.T.conj()
    return WS_symm

#compute WS symmetric
TMs = TM_modes
# TMs = evolutionTMs

center_TM_index = 24
step = 4
WS_symm = get_WS_symm(TMs[center_TM_index - step], 
                      TMs[center_TM_index], 
                      TMs[center_TM_index + step])
plt.figure(),plt.imshow(np.abs(WS_symm))

[0.00276899 0.00268304 0.00263523 0.00261299 0.00257245 0.00255955
 0.00253043 0.0025137  0.0025039  0.00247709]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x7fdfafea3668>)

In [31]:
# def pca_inv(H,n):
#     u,s,vh = np.linalg.svd(H)
#     print(s[:10])
#     s2 = np.zeros_like(s)
#     s2[:n] = 1./s[:n]
#     return vh.transpose().conjugate() @ np.diag(s2) @ u.transpose().conjugate() 

# def get_WS(Hm1,H0,H1):
# #     H1 = remove_global_phase(H1)
# #     H2 = remove_global_phase(H2)
# #     H0 = remove_global_phase(H0)
#     dH = (H1 - Hm1)*0.5
#     WS = -1j*pca_inv(H0,100)@dH
# #     WS = -1j * np.linalg.pinv(H1,rcond = 0.5) @ dH
# #     WS = -1j * H1.transpose().conjugate() @ dH
# #     u,s,v = np.linalg.svd(np.linalg.pinv(TM1,rcond = 0.5))
# #     plt.figure(),plt.plot(s,'+')
#     return WS

# def get_WS(H0,H1,H2,Hm1, Hm2):
# #     H1 = remove_global_phase(H1)
# #     H2 = remove_global_phase(H2)
# #     H0 = remove_global_phase(H0)
# #     dH = (H2 - H0)*0.5
#     c = np.array([-1,8,-8,1])/12
#     dH = c[0]*H2 + c[1]*H1+c[2]*Hm1+c[3]*Hm2
#     WS = -1j*pca_inv(H1,110)@dH
# #     WS = -1j * np.linalg.pinv(H1,rcond = 0.5) @ dH
# #     WS = -1j * H1.transpose().conjugate() @ dH
# #     u,s,v = np.linalg.svd(np.linalg.pinv(TM1,rcond = 0.5))
# #     plt.figure(),plt.plot(s,'+')
#     return WS


# def get_WS_symm(*args,**kwargs):
#     WS = get_WS(*args, **kwargs)
#     WS_symm = WS + WS.T.conj()
#     return WS_symm

# center_TM_index = 24
# step = 1

# WS_symm = get_WS_symm(TM_modes[center_TM_index], 
#                       TM_modes[center_TM_index + step], 
#                       TM_modes[center_TM_index + 2*step], 
#                       TM_modes[center_TM_index - step],
#                       TM_modes[center_TM_index - 2*step])

# plt.figure(),plt.imshow(np.abs(WS_symm))

In [11]:
a,b = np.linalg.eig(WS_symm)
a = a[np.argsort(a)]
b = b[np.argsort(a),:]

# plt.figure()
# plt.plot(a)

In [12]:
def get_out_states(in_modes, i0, i1):
#     in_pix = modes_in_full.transpose()@in_modes
#     in_pix_p1 = in_pix[:n**2].reshape([n]*2)
#     in_pix_p2 = in_pix[n**2:].reshape([n]*2)
    output_state = TMs[i0] @ in_modes
    out_pix = modes_out_full.transpose()@output_state
    out_pix_p1_0 = out_pix[:m**2].reshape([m]*2)
    out_pix_p2_0 = out_pix[m**2:].reshape([m]*2)
#     output_states_WS = [TMs[i] @ state_WS for i in range(len(TMs))]
    output_state = TMs[i1] @ in_modes
    out_pix = modes_out_full.transpose()@output_state
    out_pix_p1_f = out_pix[:m**2].reshape([m]*2)
    out_pix_p2_f = out_pix[m**2:].reshape([m]*2)
    
    return out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f

def get_correlation_state(in_modes, indexes, index_reference):
    corr_vector = []
    f = lambda x: np.abs(x)**2
#     f = lambda x: x
    for ind in indexes:
        out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(in_modes, i0 = index_reference, i1 = ind)
        out_0 = np.concatenate([out_pix_p1_0,out_pix_p2_0])
        out_f = np.concatenate([out_pix_p1_f,out_pix_p2_f])
#         print(out.shape)
        corr_vector.append(complex_correlation(f(out_0),f(out_f)))
    return corr_vector
        

In [34]:
Nmodes = TM_modes[0].shape[0]//2
n_eig_to_show = 2*Nmodes
eig_step = 1
# index_eig =  list(range(0,n_eig_to_show,eig_step)) + list(range(2*Nmodes-n_eig_to_show,2*Nmodes,eig_step))
index_eig =  list(range(0,n_eig_to_show,eig_step))
index_reference = center_TM_index

index_modes_to_show = list(range(1))

indexes = list(range(len(TMs)))

correlations_WS = []
for ind in index_eig:
    state_WS = b[:,ind] # use WS eigen vector
#     output_states_WS = [TMs[i] @ state_WS for i in range(len(TMs))]
#     correlations_WS.append([complex_correlation(output_states_WS[i],output_states_WS[index_reference]) for i in range(len(TMs))])
    correlations_WS.append(get_correlation_state(state_WS, 
                                                 indexes = indexes, 
                                                 index_reference = index_reference))

correlations_mode = []
for ind in index_modes_to_show:
    state = np.zeros(110, dtype = np.complex) # use the modes 
    state[55+ind] = 1
#     state[ind] = 1
    # state[56] = 1
    # output_states_mode = [TMs[i] @ state for i in range(len(TMs))]
    # correlations_mode = [complex_correlation(output_states_mode[i],output_states_mode[index_reference]) for i in range(len(TMs))]
    correlations_mode.append(get_correlation_state(state, indexes = indexes, index_reference = index_reference))
# output_states_optim = [TM_modes[i] @ state_retard_optimization for i in range(len(TM_modes))]
# correlations_optim = [checkComplexCorrelation(output_states_optim[i],output_states_optim[0]) for i in range(len(TM_modes))]

correlations_rand_avg = np.zeros(len(TMs))
for j in range(20):
    rand_vec = (np.random.rand(110)-0.5) + 1j * (np.random.rand(110)-0.5)
    output_states_rand = [TMs[i] @ rand_vec for i in range(len(TMs))]
    correlations_rand = np.array([complex_correlation(output_states_rand[i],output_states_rand[index_reference]) for i in range(len(TMs))])
    correlations_rand_avg += correlations_rand
correlations_rand_avg /= 20

In [35]:
max_ind = np.argmax([c[-1] for c in correlations_WS])
np.min(correlations_WS[max_ind][19:])

0.9458800940705596

In [37]:
i0 = 15
index_dx0 = 17
plt.close('all')
plt.figure()
dx = (np.array(param['Translation']['steps_pressure'])-param['Translation']['steps_pressure'][index_dx0])*1e3
cmap = get_color_map(n = n_eig_to_show)

# plt.figure(),plt.plot(dx[i0:],correlations_mode[i0:], color = 'darkgreen', marker='+')
for i,corr_WS in enumerate(correlations_WS):
    plt.plot(dx[i0:],corr_WS[i0:],color = cmap(i))
    
for i,corr_WS in enumerate(correlations_mode):
    plt.plot(dx[i0:],correlations_mode[i][i0:], color = 'darkgreen', marker='+')

plt.plot(dx[i0:],correlations_rand_avg[i0:],'b+-')

all_min = plt.gca().get_ylim()[0]
plt.vlines(dx[center_TM_index],all_min,1., 'r', linestyles = 'dashed')

# plt.plot(x,correlations_optim[i0:],'m+')
# plt.legend(['Fundamental mode', 'random mode', 'WS eigenvector','optimization'])
plt.legend(['Fondamental mode', 'Rnd wavefront', 'WS'])
plt.xlabel('Deformation in microns')
plt.ylabel('Correlation')

plt.savefig('WS_corr.svg')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
dx[-1]

70.00000000000028

In [39]:
in_state = np.zeros(110) # use the modes 
in_state[0] = 1

# to_img = colorize
to_img = lambda x: np.abs(x)**2
cmap = 'gray'

out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(in_state,
                                                                       i0 = index_dx0,
                                                                       i1 = -1)
out_0 = to_img(np.concatenate([out_pix_p1_0,out_pix_p2_0]))
out_f = to_img(np.concatenate([out_pix_p1_f,out_pix_p2_f]))
corr = complex_correlation(out_0,out_f)

plt.figure()
plt.subplot(221)
plt.imshow(to_img(out_pix_p1_0), cmap = cmap)
plt.title('Dx = 0')
plt.axis('off')
plt.subplot(223)
plt.imshow(to_img(out_pix_p2_0), cmap = cmap)
plt.axis('off')

plt.subplot(222)
plt.imshow(to_img(out_pix_p1_f), cmap = cmap)
plt.title('Dx = max')
plt.axis('off')
plt.subplot(224)
plt.imshow(to_img(out_pix_p2_f), cmap = cmap)
plt.axis('off')

plt.suptitle(f'Pearson correlation coefficient = {corr:.2f}%')
plt.savefig('fundamental_mode.svg')

rand_vec = (np.random.rand(110)-0.5) + 1j * (np.random.rand(110)-0.5)

out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f = get_out_states(rand_vec,
                                                                       i0 = index_dx0,
                                                                       i1 = -1)
out_0 = to_img(np.concatenate([out_pix_p1_0,out_pix_p2_0]))
out_f = to_img(np.concatenate([out_pix_p1_f,out_pix_p2_f]))
corr = complex_correlation(out_0,out_f)

plt.figure()
plt.subplot(221)
plt.imshow(to_img(out_pix_p1_0), cmap = cmap)
plt.title('Dx = 0')
plt.axis('off')
plt.subplot(223)
plt.imshow(to_img(out_pix_p2_0), cmap = cmap)
plt.axis('off')

plt.subplot(222)
plt.imshow(to_img(out_pix_p1_f), cmap = cmap)
plt.title('Dx = max')
plt.axis('off')
plt.subplot(224)
plt.imshow(to_img(out_pix_p2_f), cmap = cmap)
plt.axis('off')

plt.suptitle(f'Pearson correlation coefficient = {corr:.2f}%')
plt.savefig('rand.svg')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
max_ind = np.argmax([c[-1] for c in correlations_WS])
cmap = 'gray'
# to_img = colorize
to_img = lambda x: np.abs(x)**2

for ind in list(range(6))+[index_eig[max_ind]]:
    state_WS = b[:,ind] # use WS eigen vector
    out_pix_p1_0, out_pix_p2_0, out_pix_p1_f, out_pix_p2_f =  get_out_states(state_WS,
                                                                   i0 = index_dx0,
                                                                   i1 = -1)
    
    max_1 = np.max([np.max(to_img(out_pix_p1_0)),np.max(to_img(out_pix_p1_f))])
    max_2 = np.max([np.max(to_img(out_pix_p2_0)),np.max(to_img(out_pix_p2_f))])
    
    plt.figure()
    plt.subplot(221)
    plt.imshow(to_img(out_pix_p1_0), vmax = max_1, cmap = cmap)
    plt.title('Input')
    plt.axis('off')
    plt.subplot(223)
    plt.imshow(to_img(out_pix_p2_0), vmax = max_2, cmap = cmap)
    plt.axis('off')
    
    plt.subplot(222)
    plt.imshow(to_img(out_pix_p1_f), vmax = max_1, cmap = cmap)
    plt.title('Output')
    plt.axis('off')
    plt.subplot(224)
    plt.imshow(to_img(out_pix_p2_f), vmax = max_2, cmap = cmap)
    plt.axis('off')
    
    plt.suptitle(rf'Corr for $\Delta x$ max = {correlations_WS[ind][-1]:.2f}')
    plt.savefig(f'WS_mode_{ind}.svg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
output_state_WS = TMs[-1] @ state_WS
output_state_WS.shape
modes_out_full.transpose()@output_states_WS

NameError: name 'output_states_WS' is not defined